#### Python Libraries 

In [1]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import statsmodels.formula.api as smf
import statsmodels.api         as sm
import rpy2
from sklearn.utils import shuffle
from preprocess import pp_transforms, pp_tests, pp_processes
from scripts import arima, sem, holt, holt_winters
from models import AutoRegressive
from tqdm import tqdm
warnings.filterwarnings("ignore")

# Display all columns
pd.set_option('display.max_columns', None)

# Use a ggplot style for graphics
plt.style.use('ggplot')

#### R packages

In [3]:
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, numpy2ri
import rpy2.robjects as ro 

# Activate automatic conversion
pandas2ri.activate()
numpy2ri.activate()

# R libraries
tsintermittent, tscount, pscl = importr('tsintermittent'), importr('tscount'), importr('pscl')
base, ZIM, forecast = importr('base'), importr('ZIM'), importr('forecast')

# R functions 
tsb, crost = tsintermittent.tsb, tsintermittent.crost
tsglm, predict_tsglm = tscount.tsglm, tscount.predict_tsglm

# R objects 
ts=ro.r('ts')

### Load and Sample Data

In [2]:
# Load data
files = ['data/calendar.csv', 'data/sales_train_validation.csv', 'data/sell_prices.csv', 'data/dt_sales.csv']
data = [pd.read_csv(f) for f in files]
dt_calendar, dt_sales_train, dt_prices, dt_sales_zeros = data

### Count Regression Models 

In [4]:
# Bin 4 and Bin 5 items 
bins = ['perc_bin_4', 'perc_bin_5']

# MASE 
def MASE(train, test, pred):
    
    # Numerator
    numerator = test - pred
    
    # Denominator
    shift = train.shift(1)
    denominator = train - shift
    denominator = denominator.dropna()
    
    return np.mean(np.abs(numerator/denominator))


# Forecast 
def forecast_tsb(data, steps):
    
    data = data[-90:].astype(int)
    
    # Control the variance 
    data_log = np.log(data + 1)
    
    # To avoid R-error
    data_log = ro.FloatVector(data_log)
    
    # Fit a Croston modified (TSB)
    try:
        mdl_tsb = tsb(ts(data_log), h=steps, cost="mar")
        pred_tsb =  mdl_tsb.rx2('frc.out')
    except rpy2.rinterface.RRuntimeError as e:
        print(e)
        
    # Undo logarithmic transformation   
    pred_tsb = np.array(pred_tsb)
    pred_tsb = np.exp(pred_tsb) - 1
    
    return pred_tsb, data 

def forecast_tsglm(data, steps):
    
    data = data[-90:].astype(int)
    
    # Control the variance 
    data_log = np.log(data + 1)
    
    # To avoid R-error
    data_log = ro.FloatVector(data_log)
    
    # Fit an INGARCH model 
    try:
        mdl_tsglm = tsglm(ts(train_log), model=ro.r('list(past_obs = 4, past_mean = 2)'), link="log", distr = "nbinom")
        pred_tsglm = predict_tsglm(mdl_tsglm, n_ahead = steps)
        pred_tsglm, model_glm = pred_tsglm.rx2('pred'), mdl_tsglm.rx2('model')
    except rpy2.rinterface.RRuntimeError as e:
        print(e)
        
    # Undo logarithmic transformation   
    pred_tsglm = np.array(pred_tsglm)
    pred_tsglm = np.exp(pred_tsglm) - 1
    
    return pred_tsglm, data

In [ ]:
# Reset warnings 
warnings.resetwarnings()

count_mdl, invalid_skus  = {}, []
df = dt_sales_zeros[dt_sales_zeros.perc_zeros_bin.isin(bins)]

for index,row in tqdm(df.iterrows(), total=df.shape[0]):
    item_id, since_d1, zero_bin = row[0], row[-3], row[-1]
    
    # Split data
    sales = row[6+since_d1:-5]
    train, test = sales[-90:-30].astype(int), sales[-30:-1].astype(int)
    
    # Control the variance 
    train_log = np.log(train + 1)
    
    # To avoid R-error
    train_log = ro.FloatVector(train_log)
    
    # Fit a Croston modified (TSB)
    try:
        mdl_tsb = tsb(ts(train_log), h=29, cost="mar")
        pred_tsb, model_tsb = mdl_tsb.rx2('frc.out'), mdl_tsb.rx2('model')
    except rpy2.rinterface.RRuntimeError as e:
        invalid_skus.append(item_id)
        print(e)
        continue
        
    # Fit an INGARCH model 
    try:
        mdl_tsglm = tsglm(ts(train_log), model=ro.r('list(past_obs = 4, past_mean = 2)'), link="log", distr = "nbinom")
        pred_tsglm = predict_tsglm(mdl_tsglm, n_ahead = 29)
        pred_tsglm, model_glm = pred_tsglm.rx2('pred'), mdl_tsglm.rx2('model')
    except rpy2.rinterface.RRuntimeError as e:
        invalid_skus.append(item_id)
        print(e)
        continue
    
    # Undo logarithmic transformation   
    pred_tsb, pred_tsglm = np.array(pred_tsb), np.array(pred_tsglm)
    pred_tsb, pred_tsglm = np.exp(pred_tsb) - 1, np.exp(pred_tsglm) - 1
    
    # MASE
    mase_tsb = MASE(train, test, pred_tsb)
    mase_tsglm = MASE(train, test, pred_tsglm)
    
    # Model selection 
    if mase_tsb < mase_tsglm:
        point_forecast, data = forecast_tsb(sales,28)
        model = "TSB"
    else:
        point_forecast, data = forecast_tsglm(sales,28)
        model = "TSGLM"
    
    count_mdl[item_id] = point_forecast